In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
! pip install scikit-optimize
# report scikit-optimize version number
import skopt
print('skopt %s' % skopt.__version__)
from skopt.space import Integer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding, Attention
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K
import argparse

skopt 0.10.1


In [2]:
#data from kaggle dataset: "Prediction of music genre"
data1 = pd.read_csv("~/Downloads/music_genre.csv")
#data from kaggle data set: "Spotify Tracks Dataset"
data2 = pd.read_csv("~/Downloads/dataset.csv")

In [3]:
#because all the column values are none values, drop those rows
before_drop_na = len(data1)
data1 = data1.dropna()
after_drop_na = len(data1)
print("number of data dropped:", before_drop_na-after_drop_na)

number of data dropped: 5


In [4]:
#make key, mode into One Hot encoding
#music_genre into label encoding
onehot =OneHotEncoder(sparse=False)
onehot.fit(data1[['key', 'mode']])
onehotencoded = onehot.transform(data1[['key', 'mode']])
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(data1[['key', 'mode']].columns) for cat in onehot.categories_[i]]
onehotdf = pd.DataFrame(onehotencoded, columns=categorical_columns)
data1 = pd.concat([data1.reset_index(), onehotdf.reset_index()], axis=1)
labelencoder = LabelEncoder()
data1['key_num'] = labelencoder.fit_transform(data1['key'])
data1['mode_num'] = labelencoder.fit_transform(data1['mode'])
data1['music_genre_num'] = labelencoder.fit_transform(data1['music_genre'])
data1

/Users/mkobayashi/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,index,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,...,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor,key_num,mode_num,music_genre_num
0,0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,5
1,1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5,1,5
2,2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,11,0,5
3,3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,0,5
4,4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,9,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,50000,58878.0,BEXEY,GO GETTA,59.0,0.03340,0.913,-1.0,0.574,0.00000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,0,6
49996,50001,43557.0,Roy Woods,Drama (feat. Drake),72.0,0.15700,0.709,251860.0,0.362,0.00000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,0,6
49997,50002,39767.0,Berner,Lovin' Me (feat. Smiggz),51.0,0.00597,0.693,189483.0,0.763,0.00000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5,0,6
49998,50003,57944.0,The-Dream,Shawty Is Da Shit,65.0,0.08310,0.782,262773.0,0.472,0.00000,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,10,1,6


In [5]:
#not including tempo
data1_edit = data1.drop(columns = ["index", "instance_id", "artist_name", "track_name", "key", "mode", "tempo", "obtained_date", "music_genre", "key_num", "mode_num", "music_genre_num"])
features = data1_edit.columns

In [6]:
features

Index(['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence',
       'key_A', 'key_A#', 'key_B', 'key_C', 'key_C#', 'key_D', 'key_D#',
       'key_E', 'key_F', 'key_F#', 'key_G', 'key_G#', 'mode_Major',
       'mode_Minor'],
      dtype='object')

In [7]:
# train 80%, val 20%, test 20%
X_train1b, X_test1b, y_train1b, y_test1b = train_test_split(data1[features], data1["music_genre_num"], test_size=0.2, random_state=1)
X_train1b, X_val1b, y_train1b, y_val1b = train_test_split(X_train1b, y_train1b, test_size=0.25, random_state=1)

In [8]:
model1b = Sequential()
model1b.add(LSTM(units=256,input_shape=(24, 1)))
model1b.add(Dense(10, activation = "softmax"))

model1b.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history1b = model1b.fit(X_train1b, y_train1b, validation_data=(X_val1b, y_val1b), batch_size=32, epochs=100)

Epoch 1/100
938/938 [==============================] - 36s 37ms/step - loss: 1.7101 - accuracy: 0.3301 - val_loss: 1.5566 - val_accuracy: 0.3808
Epoch 2/100
938/938 [==============================] - 46s 49ms/step - loss: 1.5496 - accuracy: 0.3897 - val_loss: 1.5348 - val_accuracy: 0.4005
Epoch 3/100
938/938 [==============================] - 49s 52ms/step - loss: 1.5111 - accuracy: 0.4052 - val_loss: 1.4906 - val_accuracy: 0.4132
Epoch 4/100
938/938 [==============================] - 50s 53ms/step - loss: 1.4403 - accuracy: 0.4322 - val_loss: 1.3924 - val_accuracy: 0.4471
Epoch 5/100
938/938 [==============================] - 51s 54ms/step - loss: 1.3894 - accuracy: 0.4581 - val_loss: 1.3219 - val_accuracy: 0.4897
Epoch 6/100
938/938 [==============================] - 44s 47ms/step - loss: 1.2892 - accuracy: 0.5016 - val_loss: 1.4262 - val_accuracy: 0.4614
Epoch 7/100
938/938 [==============================] - 44s 47ms/step - loss: 1.2175 - accuracy: 0.5244 - val_loss: 1.2659 - val_ac

In [9]:
print("Training Accuracy:", model1b.evaluate(X_train1b, y_train1b))
print("Test Accuracy:", model1b.evaluate(X_test1b, y_test1b))

938/938 [==============================] - 19s 20ms/step - loss: 0.1733 - accuracy: 0.9367
Training Accuracy: [0.1732586920261383, 0.9366666674613953]
313/313 [==============================] - 5s 16ms/step - loss: 2.6673 - accuracy: 0.4794
Test Accuracy: [2.667311906814575, 0.47940000891685486]


In [10]:
print(classification_report(y_test1b, np.argmax(model1b.predict(X_test1b, 1), axis =-1)))

10000/10000 [==============================] - 49s 5ms/step
              precision    recall  f1-score   support

           0       0.32      0.33      0.32      1005
           1       0.70      0.65      0.68      1057
           2       0.47      0.43      0.45       985
           3       0.77      0.78      0.77       967
           4       0.41      0.46      0.44       969
           5       0.54      0.54      0.54      1029
           6       0.37      0.33      0.35      1012
           7       0.43      0.41      0.42       996
           8       0.34      0.37      0.35       982
           9       0.45      0.48      0.47       998

    accuracy                           0.48     10000
   macro avg       0.48      0.48      0.48     10000
weighted avg       0.48      0.48      0.48     10000



In [11]:
model1b2 = Sequential()
model1b2.add(LSTM(units=128,input_shape=(24, 1)))
model1b2.add(Dense(10, activation = "softmax"))

model1b2.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history1b2 = model1b2.fit(X_train1b, y_train1b, validation_data=(X_val1b, y_val1b), batch_size=32, epochs=100)

Epoch 1/100
938/938 [==============================] - 39s 40ms/step - loss: 1.7809 - accuracy: 0.3041 - val_loss: 1.5648 - val_accuracy: 0.3789
Epoch 2/100
938/938 [==============================] - 36s 39ms/step - loss: 1.5610 - accuracy: 0.3852 - val_loss: 1.5273 - val_accuracy: 0.3966
Epoch 3/100
938/938 [==============================] - 34s 36ms/step - loss: 1.5243 - accuracy: 0.3980 - val_loss: 1.5191 - val_accuracy: 0.3872
Epoch 4/100
938/938 [==============================] - 39s 42ms/step - loss: 1.5009 - accuracy: 0.4077 - val_loss: 1.4972 - val_accuracy: 0.3962
Epoch 5/100
938/938 [==============================] - 33s 35ms/step - loss: 1.4462 - accuracy: 0.4292 - val_loss: 1.4057 - val_accuracy: 0.4421
Epoch 6/100
938/938 [==============================] - 43s 46ms/step - loss: 1.3937 - accuracy: 0.4530 - val_loss: 1.4025 - val_accuracy: 0.4676
Epoch 7/100
938/938 [==============================] - 43s 46ms/step - loss: 1.3206 - accuracy: 0.4858 - val_loss: 1.4160 - val_ac

In [12]:
print("Training Accuracy:", model1b2.evaluate(X_train1b, y_train1b))
print("Test Accuracy:", model1b2.evaluate(X_test1b, y_test1b))

938/938 [==============================] - 8s 8ms/step - loss: 0.5711 - accuracy: 0.7712
Training Accuracy: [0.5711295008659363, 0.7712000012397766]
313/313 [==============================] - 3s 8ms/step - loss: 1.7273 - accuracy: 0.5023
Test Accuracy: [1.7272535562515259, 0.5023000240325928]


In [13]:
scaler1 = preprocessing.MinMaxScaler()
scaled_feat1 = scaler1.fit_transform(data1[features])

X_train2, X_test2, y_train2, y_test2 = train_test_split(scaled_feat1, data1["music_genre_num"], test_size=0.2, random_state=1)
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size=0.25, random_state=1)

In [14]:
model2 = Sequential()
#forget gates = 0, paththrough gate = 1
model2.add(LSTM(units=128,input_shape=(24, 1)))
model2.add(Dense(10, activation = "softmax"))

#bidirectional, attention block (inform importance of other positions)- layers.attention

model2.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history2 = model2.fit(X_train2, y_train2, validation_data=(X_val2, y_val2), batch_size=32, epochs=100)

Epoch 1/100
938/938 [==============================] - 35s 35ms/step - loss: 2.1250 - accuracy: 0.2064 - val_loss: 2.0174 - val_accuracy: 0.2521
Epoch 2/100
938/938 [==============================] - 28s 30ms/step - loss: 1.7712 - accuracy: 0.3427 - val_loss: 1.5663 - val_accuracy: 0.4027
Epoch 3/100
938/938 [==============================] - 31s 33ms/step - loss: 1.4581 - accuracy: 0.4423 - val_loss: 1.4072 - val_accuracy: 0.4596
Epoch 4/100
938/938 [==============================] - 27s 29ms/step - loss: 1.3974 - accuracy: 0.4649 - val_loss: 1.3714 - val_accuracy: 0.4668
Epoch 5/100
938/938 [==============================] - 25s 27ms/step - loss: 1.3587 - accuracy: 0.4782 - val_loss: 1.3226 - val_accuracy: 0.4858
Epoch 6/100
938/938 [==============================] - 24s 26ms/step - loss: 1.3214 - accuracy: 0.4915 - val_loss: 1.3348 - val_accuracy: 0.4762
Epoch 7/100
938/938 [==============================] - 29s 31ms/step - loss: 1.2893 - accuracy: 0.4988 - val_loss: 1.2745 - val_ac

In [15]:
print("Training Accuracy:", model2.evaluate(X_train2, y_train2))
print("Test Accuracy:", model2.evaluate(X_test2, y_test2))

938/938 [==============================] - 4s 4ms/step - loss: 0.5500 - accuracy: 0.7856
Training Accuracy: [0.5499528050422668, 0.7856000065803528]
313/313 [==============================] - 2s 5ms/step - loss: 1.5875 - accuracy: 0.5095
Test Accuracy: [1.5874824523925781, 0.5095000267028809]


In [16]:
model3 = Sequential()
#forget gates = 0, paththrough gate = 1

model3.add((Bidirectional(LSTM(units=128, input_shape=(24, 1)))))
model3.add(Dense(10, activation = "softmax"))

#bidirectional, attention block (inform importance of other positions)- layers.attention

model3.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history3 = model3.fit(np.expand_dims(X_train2, 1), y_train2, validation_data=(np.expand_dims(X_val2, 1), y_val2), batch_size=32, epochs=100)

Epoch 1/100
938/938 [==============================] - 180s 188ms/step - loss: 1.6104 - accuracy: 0.4168 - val_loss: 1.3468 - val_accuracy: 0.5060
Epoch 2/100
938/938 [==============================] - 3s 3ms/step - loss: 1.3037 - accuracy: 0.5108 - val_loss: 1.2940 - val_accuracy: 0.5115
Epoch 3/100
938/938 [==============================] - 3s 3ms/step - loss: 1.2718 - accuracy: 0.5196 - val_loss: 1.2824 - val_accuracy: 0.5172
Epoch 4/100
938/938 [==============================] - 2s 2ms/step - loss: 1.2557 - accuracy: 0.5258 - val_loss: 1.2722 - val_accuracy: 0.5182
Epoch 5/100
938/938 [==============================] - 2s 2ms/step - loss: 1.2466 - accuracy: 0.5290 - val_loss: 1.2662 - val_accuracy: 0.5224
Epoch 6/100
938/938 [==============================] - 2s 2ms/step - loss: 1.2403 - accuracy: 0.5330 - val_loss: 1.2570 - val_accuracy: 0.5326
Epoch 7/100
938/938 [==============================] - 3s 3ms/step - loss: 1.2343 - accuracy: 0.5327 - val_loss: 1.2524 - val_accuracy: 0.

In [17]:
print("Training Accuracy:", model3.evaluate(np.expand_dims(X_train2, 1), y_train2))
print("Test Accuracy:", model3.evaluate(np.expand_dims(X_test2, 1), y_test2))

938/938 [==============================] - 1s 1ms/step - loss: 1.0345 - accuracy: 0.6003
Training Accuracy: [1.0344985723495483, 0.6003333330154419]
313/313 [==============================] - 0s 1ms/step - loss: 1.1183 - accuracy: 0.5665
Test Accuracy: [1.1183393001556396, 0.5665000081062317]


In [18]:
print(classification_report(y_test2, np.argmax(model3.predict(np.expand_dims(X_test2, 1)), axis =-1)))

313/313 [==============================] - 1s 767us/step
              precision    recall  f1-score   support

           0       0.42      0.40      0.41      1005
           1       0.73      0.71      0.72      1057
           2       0.51      0.59      0.55       985
           3       0.83      0.86      0.84       967
           4       0.56      0.46      0.51       969
           5       0.70      0.54      0.61      1029
           6       0.45      0.44      0.45      1012
           7       0.53      0.52      0.53       996
           8       0.44      0.49      0.47       982
           9       0.53      0.65      0.59       998

    accuracy                           0.57     10000
   macro avg       0.57      0.57      0.57     10000
weighted avg       0.57      0.57      0.57     10000



In [19]:
model4 = Sequential()
#forget gates = 0, paththrough gate = 1

model4.add(Bidirectional(LSTM(units=128, input_shape=(24, 1))))
model4.add(Dropout(0.1))
model4.add(Dense(10, activation = "softmax"))

#bidirectional, attention block (inform importance of other positions)- layers.attention

model4.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history4 = model4.fit(np.expand_dims(X_train2, 1), y_train2, validation_data=(np.expand_dims(X_val2, 1), y_val2), batch_size=32, epochs=100)

Epoch 1/100
938/938 [==============================] - 7s 4ms/step - loss: 1.6191 - accuracy: 0.4173 - val_loss: 1.3493 - val_accuracy: 0.4950
Epoch 2/100
938/938 [==============================] - 5s 5ms/step - loss: 1.3094 - accuracy: 0.5102 - val_loss: 1.2993 - val_accuracy: 0.5126
Epoch 3/100
938/938 [==============================] - 3s 3ms/step - loss: 1.2751 - accuracy: 0.5207 - val_loss: 1.2790 - val_accuracy: 0.5233
Epoch 4/100
938/938 [==============================] - 2s 2ms/step - loss: 1.2617 - accuracy: 0.5239 - val_loss: 1.2672 - val_accuracy: 0.5253
Epoch 5/100
938/938 [==============================] - 2s 2ms/step - loss: 1.2543 - accuracy: 0.5270 - val_loss: 1.2644 - val_accuracy: 0.5256
Epoch 6/100
938/938 [==============================] - 4s 4ms/step - loss: 1.2478 - accuracy: 0.5277 - val_loss: 1.2603 - val_accuracy: 0.5309
Epoch 7/100
938/938 [==============================] - 4s 4ms/step - loss: 1.2402 - accuracy: 0.5332 - val_loss: 1.2516 - val_accuracy: 0.5325

In [20]:
print("Training Accuracy:", model4.evaluate(np.expand_dims(X_train2, 1), y_train2))
print("Test Accuracy:", model4.evaluate(np.expand_dims(X_test2, 1), y_test2))

938/938 [==============================] - 2s 2ms/step - loss: 1.0514 - accuracy: 0.5916
Training Accuracy: [1.051355242729187, 0.5916333198547363]
313/313 [==============================] - 1s 2ms/step - loss: 1.1241 - accuracy: 0.5655
Test Accuracy: [1.1240668296813965, 0.565500020980835]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3da032f1-e5ab-4726-ac09-eb3d9c053730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>